## Day 24 Lecture 2 Assignment

In this assignment, we will build our a more complex logistic regression model, this time on both numeric and categorical data. We will use the Chicago traffic crashes dataset loaded below and analyze the model generated for this dataset.

In [83]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [84]:
def missingness_summary(df, print_log=False, sort='none'):
    summary = df.apply(lambda x: x.isna().sum() / x.shape[0])
    
    if print_log == True:
        if sort == 'none':
            print(summary)
        elif sort == 'ascending':
            print(summary.sort_values())
        elif sort == 'descending':
            print(summary.sort_values(ascending=False))
        else:
            print('Invalid value for sort parameter.')
        
    return summary

In [85]:
crash_data = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/traffic_crashes_chicago.csv')

In [86]:
crash_data.head()

,RD_NO,CRASH_DATE,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,DEVICE_CONDITION,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,TRAFFICWAY_TYPE,LANE_CNT,ALIGNMENT,ROADWAY_SURFACE_COND,ROAD_DEFECT,REPORT_TYPE,CRASH_TYPE,INTERSECTION_RELATED_I,NOT_RIGHT_OF_WAY_I,HIT_AND_RUN_I,DAMAGE,DATE_POLICE_NOTIFIED,PRIM_CONTRIBUTORY_CAUSE,SEC_CONTRIBUTORY_CAUSE,STREET_NO,STREET_DIRECTION,STREET_NAME,BEAT_OF_OCCURRENCE,PHOTOS_TAKEN_I,STATEMENTS_TAKEN_I,DOORING_I,WORK_ZONE_I,WORK_ZONE_TYPE,WORKERS_PRESENT_I,NUM_UNITS,MOST_SEVERE_INJURY,INJURIES_TOTAL,INJURIES_FATAL,INJURIES_INCAPACITATING,INJURIES_NON_INCAPACITATING,INJURIES_REPORTED_NOT_EVIDENT,INJURIES_NO_INDICATION,INJURIES_UNKNOWN
0,JC334993,7/4/2019 22:33,45,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",REAR END,DIVIDED - W/MEDIAN BARRIER,NaN,STRAIGHT AND LEVEL,DRY,NO DEFECTS,NaN,NO INJURY / DRIVE AWAY,NaN,NaN,NaN,"OVER $1,500",7/4/2019 23:05,FOLLOWING TOO CLOSELY,NOT APPLICABLE,300,N,LAKE SHORE DR SB,114.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,JC370822,7/30/2019 10:22,30,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,TURNING,DIVIDED - W/MEDIAN (NOT RAISED),NaN,STRAIGHT AND LEVEL,DRY,NO DEFECTS,NaN,NO INJURY / DRIVE AWAY,NaN,NaN,NaN,"OVER $1,500",7/30/2019 10:25,FAILING TO YIELD RIGHT-OF-WAY,IMPROPER TURNING/NO SIGNAL,8201,S,DR MARTIN LUTHER KING JR DR,631.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,JC387098,8/10/2019 17:00,25,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,PARKED MOTOR VEHICLE,ONE-WAY,NaN,STRAIGHT AND LEVEL,DRY,NO DEFECTS,NaN,NO INJURY / DRIVE AWAY,NaN,NaN,NaN,"$501 - $1,500",8/10/2019 17:35,EQUIPMENT - VEHICLE CONDITION,NOT APPLICABLE,6747,S,CREGIER AVE,332.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,JC395195,8/16/2019 16:53,30,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,PARKED MOTOR VEHICLE,NOT DIVIDED,NaN,STRAIGHT AND LEVEL,DRY,NO DEFECTS,NaN,NO INJURY / DRIVE AWAY,NaN,NaN,Y,"$501 - $1,500",8/16/2019 16:53,UNABLE TO DETERMINE,NOT APPLICABLE,554,N,FRANKLIN ST,1831.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NO INDICATION OF INJURY,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,JC396604,8/17/2019 16:04,30,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,PARKED MOTOR VEHICLE,PARKING LOT,NaN,STRAIGHT AND LEVEL,DRY,NO DEFECTS,NaN,NO INJURY / DRIVE AWAY,NaN,NaN,Y,"$501 - $1,500",8/17/2019 18:30,UNABLE TO DETERMINE,UNABLE TO DETERMINE,3700,N,WESTERN AVE,1921.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NO INDICATION OF INJURY,0.0,0.0,0.0,0.0,0.0,1.0,0.0


First, create a binary response column by modifying the "DAMAGE" column. Consider "OVER \$1500" to be the positive class, and under \$1500 to be the negative class.

In [87]:
# answer goes here
crash_data.loc[crash_data['DAMAGE']  !=  "OVER $1,500", 'bDamaged'] = 0
crash_data.loc[crash_data['DAMAGE']  ==  "OVER $1,500", 'bDamaged'] = 1

crash_data['bDamaged'].describe()



count    372585.000000
mean          0.563418
std           0.495963
min           0.000000
25%           0.000000
50%           1.000000
75%           1.000000
max           1.000000
Name: bDamaged, dtype: float64

Using the code from Day 21, Lecture 1 as a starting point, devise an appropriate way to address missing values. You have a lot of freedom here; we will proceed by taking the following steps:

- Dropping all columns with more than 5% missing data
1- Imputing the median for numeric columns with less than 5% missing data (except for STREET_NO; imputing it in this manner would not make any sense)
- Dropping rows with missing data for categorical columns that have less than 5% missing data

In [88]:
# answer goes here
#A summary of missing variables represented as a percentage of the total missing content. 
def missingness_summary(df, print_log=False, sort='ascending'):
  s = df.isnull().sum()*100/df.isnull().count()
  s = s [s > 0]
  if sort.lower() == 'ascending':
    s = s.sort_values(ascending=True)
  elif sort.lower() == 'descending':
    s = s.sort_values(ascending=False)  
  if print_log: 
    print(s)
  
  return pd.Series(s)

suspects = missingness_summary(crash_data, False, 'descending')

drop = suspects[suspects > 5]
crash_data.drop(columns=drop.keys(), axis=1, inplace=True)

In [89]:
suspects = missingness_summary(crash_data, False, 'descending')

In [90]:
fill_list = crash_data[suspects.keys()].select_dtypes(include='number').columns
for col in fill_list: 
  crash_data.loc[:, col] = crash_data.loc[:, col].fillna(crash_data.loc[:, col].mean())

In [91]:
suspects = missingness_summary(crash_data, False, 'descending')
crash_data.dropna(subset=suspects.keys(), inplace=True)

In [92]:
suspects = missingness_summary(crash_data, False, 'descending')
suspects

Series([], dtype: float64)

Finally, choose a few numeric and categorical features (2-3 of each) to include in the model. (You can definitely include more than this, but too many features, especially categorical ones, will most likely lead to convergence issues). One hot encode the chosen categorical features, being sure to omit one of the categories (which will serve as a "reference" level) to avoid perfect multicollinearity.

Again, you have a lot of freedom here; we will proceed with the following features, dropping the most commonly occurring category for the two categorical variables ("CLEAR" for weather, "REAR END" for first crash type):
POSTED_SPEED_LIMIT, WEATHER_CONDITION, INJURIES_TOTAL, FIRST_CRASH_TYPE

In [93]:
# answer goes here
features = ['POSTED_SPEED_LIMIT', 'INJURIES_TOTAL']
dum1 = 'WEATHER_CONDITION'
dum2 = 'FIRST_CRASH_TYPE'
df = crash_data[features]
one_hot = pd.get_dummies(crash_data[dum1], drop_first=True)
two_hot = pd.get_dummies(crash_data[dum2], drop_first=True)


In [94]:
one_hot.drop(axis=1, columns='CLEAR', inplace=True)
two_hot.drop(axis=1, columns='REAR END', inplace=True)

In [95]:
hots = one_hot.join(two_hot)
df = df.join(hots)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 362483 entries, 6 to 372584
Data columns (total 27 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   POSTED_SPEED_LIMIT            362483 non-null  int64  
 1   INJURIES_TOTAL                362483 non-null  float64
 2   CLOUDY/OVERCAST               362483 non-null  uint8  
 3   FOG/SMOKE/HAZE                362483 non-null  uint8  
 4   FREEZING RAIN/DRIZZLE         362483 non-null  uint8  
 5   OTHER                         362483 non-null  uint8  
 6   RAIN                          362483 non-null  uint8  
 7   SEVERE CROSS WIND GATE        362483 non-null  uint8  
 8   SLEET/HAIL                    362483 non-null  uint8  
 9   SNOW                          362483 non-null  uint8  
 10  UNKNOWN                       362483 non-null  uint8  
 11  ANIMAL                        362483 non-null  uint8  
 12  FIXED OBJECT                  362483 non-nul

Split the data into train and test, with 80% training and 20% testing. By default, the LR output from statsmodels does not include an intercept terms; add a constant column to the training data so that an intercept term is calculated for the LR model (hint: sm.add_constant() is a useful function to accomplish this).

In [96]:
# answer goes here
from sklearn.model_selection import train_test_split
X = df
y = crash_data['bDamaged']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


In [97]:
import statsmodels.api as sm

X_train_const = sm.add_constant(X_train)

sm_model = sm.Logit(y_train, X_train_const).fit()

print(sm_model.summary())

Optimization terminated successfully.
         Current function value: 0.661652
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:               bDamaged   No. Observations:               289986
Model:                          Logit   Df Residuals:                   289958
Method:                           MLE   Df Model:                           27
Date:                Thu, 15 Oct 2020   Pseudo R-squ.:                 0.03481
Time:                        20:10:45   Log-Likelihood:            -1.9187e+05
converged:                       True   LL-Null:                   -1.9879e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
const                           -0.0855      0.018     -4.698      0.000

Fit the logistic regression model using the statsmodels package and print out the coefficient summary. Which variables (in particular, which categories of our categorical variables) appear to be the most important, and what effect do they have on the probability of a crash resulting in $1500 or more in damages?

In [98]:
# answer goes here
sm_model = sm.Logit(y_train, X_train_const).fit()
sm_model.summary()

Optimization terminated successfully.
         Current function value: 0.661652
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               bDamaged   No. Observations:               289986
Model:                          Logit   Df Residuals:                   289958
Method:                           MLE   Df Model:                           27
Date:                Thu, 15 Oct 2020   Pseudo R-squ.:                 0.03481
Time:                        20:10:47   Log-Likelihood:            -1.9187e+05
converged:                       True   LL-Null:                   -1.9879e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
================================================================================================
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
const                           -0.0855      0.018     -4.698      0.000      -0.121      -0.050
POSTED_SPEED_LIMIT               0.0086      0.001     14.685      0.000       0.007       0.010
INJURIES_TOTAL                   0.7688      0.012     64.722      0.000       0.745       0.792
CLOUDY/OVERCAST                  0.0836      0.023      3.694      0.000       0.039       0.128
FOG/SMOKE/HAZE                   0.1913      0.091      2.106      0.035       0.013       0.369
FREEZING RAIN/DRIZZLE            0.4620      0.190      2.437      0.015       0.090       0.834
OTHER                            0.2898      0.071      4.071      0.000       0.150       0.429
RAIN                             0.1382      0.013     10.284      0.000       0.112       0.165
SEVERE CROSS WIND GATE           0.5021      0.268      1.876      0.061      -0.022       1.027
SLEET/HAIL                       0.1892      0.100      1.883      0.060      -0.008       0.386
SNOW                             0.1509      0.021      7.142      0.000       0.109       0.192
UNKNOWN                          0.1527      0.019      8.034      0.000       0.115       0.190
ANIMAL                          -0.5589      0.154     -3.630      0.000      -0.861      -0.257
FIXED OBJECT                     0.5360      0.021     25.707      0.000       0.495       0.577
HEAD ON                          0.3773      0.046      8.270      0.000       0.288       0.467
OTHER NONCOLLISION              -0.5805      0.063     -9.240      0.000      -0.704      -0.457
OTHER OBJECT                    -0.1536      0.040     -3.845      0.000      -0.232      -0.075
OVERTURNED                       1.2686      0.257      4.938      0.000       0.765       1.772
PARKED MOTOR VEHICLE             0.0067      0.011      0.635      0.526      -0.014       0.027
PEDALCYCLIST                    -2.0133      0.039    -52.066      0.000      -2.089      -1.938
PEDESTRIAN                      -1.9392      0.031    -63.454      0.000      -1.999      -1.879
REAR TO FRONT                   -0.2116      0.067     -3.142      0.002      -0.344      -0.080
REAR TO REAR                    -0.4332      0.140     -3.083      0.002      -0.709      -0.158
REAR TO SIDE                    -0.1535      0.083     -1.847      0.065      -0.316       0.009
SIDESWIPE OPPOSITE DIRECTION    -0.1775      0.032     -5.606      0.000      -0.240      -0.115
SIDESWIPE SAME DIRECTION        -0.1614      0.011    -14.243      0.000      -0.184      -0.139
TRAIN                            1.9590      1.043      1.879      0.060      -0.085       4.003
TURNING                          0.2791      0.012     22.654      0.000       0.255       0.303
================================================================================================
"""

The data is too cluttered and confusing to provide effective and efficient answers. I can tell that there are coefficients. I know that those coefficients mean something. I don't remember what. 

I also forgot what all the categories mean and where they fall. 

There's got to be a way to do this programmatically that is both reusable and easy. 

Pedacyclist and Trains have the highest absolute coefficient, we think. Those are both in the crash type category.

Create a LogisticRegression model with sklearn. Use the .predict() method (using X_test) to get a y_pred. Create a confusion matrix comparing your actual y_test to your prediction. What do you notice about your type of error?

In [104]:
# answer goes here
from sklearn.linear_model import LogisticRegression 
logit = LogisticRegression(max_iter=1000) 
logit.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [105]:
y_pred = logit.predict(X_test)
y_pred

array([1., 1., 1., ..., 1., 1., 1.])

In [108]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, logit.predict(X_test))

array([[ 3714, 27984],
       [ 2297, 38502]])

There's a lot of positivity in this dataset. There are way more positives than there are negatives. What's the perecentage look like?